In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='arch._utility')

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm


In [3]:
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd

In [5]:
from arch import arch_model

In [6]:
import arch.__future__._utility as arch_utility
arch_utility.reindex = True


In [7]:
stock='AXISBANK.NS'
data=yf.download(stock,'2005-01-01','2023-05-05')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-01-03,37.000000,38.200001,37.000000,38.040001,32.853146,1421010
2005-01-04,38.220001,38.400002,37.400002,37.939999,32.766781,1115825
2005-01-05,37.990002,37.990002,35.509998,36.849998,31.825397,1163085
2005-01-06,36.820000,38.000000,35.400002,37.029999,31.980858,1192610
2005-01-07,36.130001,38.380001,36.130001,38.020000,32.835865,1530260
...,...,...,...,...,...,...
2023-04-27,888.000000,890.000000,877.500000,881.400024,879.590454,15992383
2023-04-28,885.099976,890.049988,853.650024,860.000000,858.234375,23004117
2023-05-02,870.000000,876.400024,864.250000,870.650024,868.862549,12947953


In [8]:
returns=np.diff(data['Adj Close'])
returns

array([ -0.08636475,  -0.94138336,   0.15546036, ...,  10.62817383,
       -10.77783203,   6.08746338])

In [9]:
import torch
import torch.nn as nn

In [10]:
import pandas as pd
import numpy as np
import arch

In [11]:
returns.shape

(4524,)

In [12]:
def garchquantile(returns):
  garch_model = arch.arch_model(returns, vol='Garch', p=1, q=1)
  garch_result = garch_model.fit(disp='off')
  garch_forecasts = garch_result.forecast(start=0)
  forecasted_volatility = np.sqrt(garch_forecasts.variance.values[-1])
  confidence_level = 0.05  # 95% confidence level
  z_score = norm.ppf(confidence_level)
  VaR = -z_score * forecasted_volatility
  return VaR

In [14]:
predictions=np.zeros(453)
for i in range(453):
  predictions[i]=(garchquantile(returns[0:4071+i]))

In [15]:
ans=returns[4071:]
ans.shape

(453,)

In [16]:
error=predictions+ans
error

array([ 14.26355104,  20.35694478,   6.99847519,   8.37802727,
        21.45154639,  32.0707907 ,  15.26823989,  19.62144055,
        12.18495548,  -1.08915134,   6.26014254,  16.67597033,
        23.868297  ,  16.07480588,  -9.75514029,   8.4408277 ,
        11.45334203,   8.44162336,  28.83095117,  34.2042395 ,
        32.78071214,  13.60639031,   9.26132399,  32.10530303,
        23.93186565,  16.81035719,  13.64247004,  19.20998097,
        18.3302971 ,  10.57574239,  13.17148008,   1.017988  ,
        11.55091116,  25.55754228,   8.13957069,  22.53166781,
        21.11797175,  46.38799994,  21.21727729,  30.63690531,
        22.16966133,  13.26172221,  20.33874871,   4.19818251,
        24.2009318 ,  12.55496765,  14.92544156,  22.31547149,
        13.00784601,  26.76918446,  20.49217992,   3.96895593,
        15.1491392 ,   8.25254787,  38.77431718,   3.13192353,
        16.47429034,   9.38107482,   9.881191  ,   1.1295439 ,
        18.87863521,  27.64273681,  20.45546894,  12.82

In [17]:
loss=0.0
for i in range(error.shape[0]):
  if(error[i]<0):
    loss=loss+-1*0.95*error[i]
  else:
    loss=loss+(1)*error[i]*(0.05)
loss/error.shape[0]

1.4079345716592235